# Azure Cognitive Search LangChain Vector Code Sample
This code demonstrates how to use Azure Cognitive Search with OpenAI and the Azure Cognitive Search LangChain Vector Store
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents==11.4.0b6` until the Langchain AzureSearch vector store integration gets updated with the latest python azure sdk updates`.

In [1]:
! pip install azure-search-documents==11.4.0b6
! pip install openai
! pip install python-dotenv
! pip install langchain

  Using cached azure_search_documents-11.4.0b6-py3-none-any.whl (306 kB)
  Attempting uninstall: azure-search-documents
    Found existing installation: azure-search-documents 11.4.0b8
    Uninstalling azure-search-documents-11.4.0b8:
      Successfully uninstalled azure-search-documents-11.4.0b8



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import required libraries and environment variables

In [14]:
# Import required libraries  
import openai
import os  
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from azure.search.documents.indexes.models import (
    SemanticSettings,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField
)


## Configure OpenAI Settings

In [3]:
# Configure environment variables  
load_dotenv()  
openai.api_type: str = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
model: str = "text-embedding-ada-002"

## Configure Vector Store Settings

In [4]:
vector_store_address: str = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")  
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
index_name: str = "langchain-vector-demo"

## Create embeddings and vector store instances
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [23]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, model=model, chunk_size=1, openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"), openai_api_type="azure" )
index_name: str = "langchain-vector-demo"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name='config',
        semantic_settings=SemanticSettings(
            default_configuration='config',
            configurations=[
                SemanticConfiguration(
                    name='config',
                    prioritized_fields=PrioritizedFields(
                        title_field=SemanticField(field_name='content'),
                        prioritized_content_fields=[SemanticField(field_name='content')],
                        prioritized_keywords_fields=[SemanticField(field_name='metadata')]
                    ))
            ])
    )


NameError: name 'search_type' is not defined

## Insert text and embeddings into vector store

In [8]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../data/state_of_the_union.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)

['YmJjNzBhNGEtMGE0OC00ODYzLWJhMzQtY2FhMTY5NWQ2OGJj',
 'MjY5MmUxODItYzA2NC00MWYzLWJhMzItNzM0ZjhjYjljNzhj',
 'NzJjZDk0MzgtY2FmNy00YWQ4LTk4YzgtYzlhZDZkMmI5YjY3',
 'YjdiYTY5OGItNGEyMC00ZWEyLTk0NzQtMGU0MmRlOGViNjRj',
 'ODJlYTgzNWEtNTY2MC00MzMzLTkzYjYtN2U0ZGVhODUzMWY5',
 'ZWI1Yzk4NTMtZmVhMy00ZWQwLTk2NjItYTlmZjFkZDdlMThk',
 'ZDBmODQzMzktNDU3Ny00MDMxLWE0MjItMmM1NTg1ZGQ4MTJm',
 'MzE1OGNhMDgtZDU3ZS00OGM3LWJlZTQtMmFjNDM2ZTA5YTJh',
 'MTBhYmE3NTItNTYxZS00N2Y5LThjNmQtYTAxZDQ3YWY2ZjY5',
 'NWRmMzNmNGUtNmYzZi00MGVhLTk4NDAtZjczODU2MWY4MGNm',
 'NmRiZGM4YzAtYzgwZi00OTk2LWIyNDEtZWZmNjc3YmJhNTcy',
 'ZjJlNzM2ZWItNWVmNC00ZTM1LWFlZGUtYjkwNDY2ZWNhZTYx',
 'NmE4MGZlYTQtODM1OC00YmRjLWIzZDgtMDRhMDU3YzRkOTEy',
 'ZTcxMTA1NmQtMTk5Ni00MTAxLWE0NzktMmJiZTkxZDYwMGQz',
 'YjJjMGNkOGEtYWQ2ZS00YTAyLWIzZTMtNTIxYzBhZDFkYTY4',
 'ZGQ0ZDMzYmMtM2JiNy00Zjg2LTg3MWQtNDBmY2FiYTk2OWIx',
 'ODNhNjQ0ZTktMzI1MS00YzRjLWEwMDAtODdkOTg3NWJlMzhl',
 'OWZmNjBjNmMtMzcyNS00ZjE4LTg4YmEtNDBkN2I1MjcyOWQ3',
 'ZGFlOTdmZDMtMTkzZC00NWRmLThlMDUtMzFhMmMzNjgy

## Perform a vector similarity search

In [21]:
# Perform a similarity search
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Perform a hybrid search

In [10]:
# Perform a hybrid search
docs = vector_store.similarity_search(
    query="What did the president say about Ketanji Brown Jackson",
    k=3, 
    search_type="hybrid"
)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Perform a Hybrid Search with Semantic re-ranking (powered by Bing)

In [60]:
# Perform a hybrid search with semantic reranking  
docs_and_scores = vector_store.semantic_hybrid_search_with_score(  
    query="What did the president say about Ketanji Brown Jackson",  
    k=3,  
)  
  
# Print the results  
for doc, score in docs_and_scores:  
    print("-" * 80)  
    answers = doc.metadata['answers']  
    if answers:  
        if answers.get('highlights'):  
            print(f"Semantic Answer: {answers['highlights']}")  
        else:  
            print(f"Semantic Answer: {answers['text']}")  
        print(f"Semantic Answer Score: {score}")  
    print("Content:", doc.page_content)  
    captions = doc.metadata['captions']
    print(f"Score: {score}") 
    if captions:  
        if captions.get('highlights'):  
            print(f"Caption: {captions['highlights']}")  
        else:  
            print(f"Caption: {captions['text']}")  
    else:  
        print("Caption not available")  


--------------------------------------------------------------------------------
Content: Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
Score: 0.03333333507180214
Caption: One of the most serious constitutional responsibilities a President has is nominating 